# General instructions

The goal of the project is to materialize a set of **exploratory workloads** over a real-world, large-scale,  open-domain KG: [WikiData](https://www.wikidata.org/wiki/Wikidata:Main_Page)

An exploratory workload  is composed by a set of queries, where each query is related to the information obtained previously.

An exploratory workload starts with a usually vague, open ended question, and does not assume the person issuing the workload has a clear understanding of the data contained in the target database or its structure.

Remeber that:

1. All the queries must run in the python notebook
2. You can use classes and properties only if you find them via a SPARQL query that must be present in the notebook
3. You do not delete useless queries. Keep everything that is synthatically valid 

```
?p <http://schema.org/name> ?name .
```
    
    is the BGP returning a human-readable name of a property or a class in Wikidata.

In [1]:
## SETUP used later

from SPARQLWrapper import SPARQLWrapper, JSON


prefixString = """
##-cc8ef16235-##
PREFIX wd: <http://www.wikidata.org/entity/> 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> 
PREFIX sc: <http://schema.org/>
"""

# select and construct queries
def run_query(queryString,verbose = True):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-03.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        results = sparql.query()
        json_results = results.convert()
        if len(json_results['results']['bindings'])==0:
            print("Empty")
            return []
        array = []
        for bindings in json_results['results']['bindings']:
            app =  [ (var, value['value'])  for var, value in bindings.items() ] 
            if verbose:
                print( app)
            array.append(app)
        if verbose:
            print(len(array))
        return array

    except Exception as e :
        print("The operation failed", e)
    
# ASk queries
def run_ask_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-03.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        return sparql.query().convert()

    except Exception as e :
        print("The operation failed", e)

# Sport Workflow Series (Basketball and NBA finals explorative search") 

Consider the following exploratory information need:

> investigate the world of basketball. Compare italian teams and US teams, country of citizenship of NBA players, and teams which won NBA finals.

## Useful URIs for the current workflow
The following are given:

| IRI           | Description   | Role      |
| -----------   | -----------   |-----------|
| `wdt:P1647`   | subproperty   | predicate |
| `wdt:P31`     | instance of   | predicate |
| `wdt:P106`    | profession    | predicate | 
| `wdt:P279`    | subclass      | predicate |
| `wdt:P27`     | nationality   | predicate |
| `wd:Q5`| Human       | node |
| `wd:Q36159`| Lebron James      | node |
| `wd:Q29545`| Kevin Durant      | node |





Also consider

```
wd:Q36159 ?p ?obj .
```

is the BGP to retrieve all **properties of Lebron James**

Please consider that when you return a resource, you should return the IRI and the label of the resource. In particular, when the task require you to identify a BGP the result set must always be a list of couples IRI - label.


The workload should:


1. Identify the BGP for basketball player.

2. Identify the BGP for basketball.

3. Identify the BGP for basketball team.

4. Return all italian basketball teams born before 1945 (the result set must be a list of triples basketball team IRI, label and year of inception).

5. Identify the BGP for NBA finals

6. Consider only the basketball teams which Kevin Durant and Lebron James played. Return the number of NBA finals won by these teams (the result set must be a list of triples basketball team IRI, label and #victories. Return a number for each basketball team).

7. Consider only NBA (National Basketball Association) basketball teams and all basketball players who played in one of these teams at least once. Also consider the country of citizenship of the players with the related continent. Return for each continent the number of players who played in an NBA basketball teams. (the result set must be triples of continent IRI, labels and #players)

8. Identify the BGP for the NBA Most Valuable Player Award (MVP) (see https://en.wikipedia.org/wiki/NBA_Most_Valuable_Player_Award)

9. Consider the players who won the NBA Most Valuable Player Award. Return for these players the number of NBA Most Valuable Player Award won in their career (the result set must be triples of player IRI, label and #victories)



## Task 1
Identify the BGP for basketball player.

In [2]:
# query example
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q36159 ?p ?obj .
   # get the label
   ?p sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('name', 'occupation')]
[('p', 'http://www.wikidata.org/prop/direct/P118'), ('name', 'league')]
[('p', 'http://www.wikidata.org/prop/direct/P1207'), ('name', 'NUKAT ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1263'), ('name', 'NNDB people ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1280'), ('name', 'CONOR.SI ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1285'), ('name', 'Munzinger Sport number')]
[('p', 'http://www.wikidata.org/prop/direct/P1344'), ('name', 'participant in')]
[('p', 'http://www.wikidata.org/prop/direct/P1411'), ('name', 'nominated for')]
[('p', 'http://www.wikidata.org/prop/direct/P1412'), ('name', 'languages spoken, written or signed')]
[('p', 'http://www.wikidata.org/prop/direct/P1417'), ('name', 'Encyclopædia Britannica Online ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1447'), ('name', 'Sports-Reference.com Olympic athlete ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1532'),

In [3]:
# Check what is Lebron James instance of
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   # bind something
   wd:Q36159 wdt:P31 ?obj .
   # get the label
   ?obj sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q5'), ('name', 'human')]
1


In [4]:
# FINAL QUERY
# Check what is Lebron James occupation: basketball player (wd:Q3665646)
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   # bind something
   wd:Q36159 wdt:P106 ?obj .
   # get the label
   ?obj sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q10798782'), ('name', 'television actor')]
[('obj', 'http://www.wikidata.org/entity/Q28389'), ('name', 'screenwriter')]
[('obj', 'http://www.wikidata.org/entity/Q3665646'), ('name', 'basketball player')]
3


>The node we are interestd in is "wd:Q3665646" with label "basketball player"

## Task 2
Identify the BGP for basketball.

In [5]:
# properties of basketball player
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q3665646 ?p ?obj .
   # get the label
   ?p sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1014'), ('name', 'Art & Architecture Thesaurus ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1036'), ('name', 'Dewey Decimal Classification')]
[('p', 'http://www.wikidata.org/prop/direct/P1296'), ('name', 'Gran Enciclopèdia Catalana ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1424'), ('name', "topic's main template")]
[('p', 'http://www.wikidata.org/prop/direct/P1709'), ('name', 'equivalent class')]
[('p', 'http://www.wikidata.org/prop/direct/P1963'), ('name', 'properties for this type')]
[('p', 'http://www.wikidata.org/prop/direct/P227'), ('name', 'GND ID')]
[('p', 'http://www.wikidata.org/prop/direct/P2347'), ('name', 'YSO ID')]
[('p', 'http://www.wikidata.org/prop/direct/P244'), ('name', 'Library of Congress authority ID')]
[('p', 'http://www.wikidata.org/prop/direct/P268'), ('name', 'Bibliothèque nationale de France ID')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('name', 'subclass of')]
[('p', 'http://www.w

In [6]:
# FINAL QUERY
# what sport (wdt:P641) is basketball player referred to 
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   # bind something
   wd:Q3665646 wdt:P641 ?obj .
   # get the label
   ?obj sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q5372'), ('name', 'basketball')]
1


>As result we got that basktball player is referred to the sport "wd:Q5372" with label "basketball"

## Task 3
Identify the BGP for basketball team.

In [7]:
# Lebron James partecipant in (wdt:P1344). Predicate found in the first query of the notebook. 
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   # bind something
   wd:Q36159 wdt:P1344 ?obj .
   # get the label
   ?obj sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q241329'), ('name', '2006 FIBA World Championship')]
[('obj', 'http://www.wikidata.org/entity/Q178595'), ('name', '2007 FIBA Americas Championship')]
[('obj', 'http://www.wikidata.org/entity/Q8558'), ('name', '2004 Summer Olympics')]
[('obj', 'http://www.wikidata.org/entity/Q8567'), ('name', '2008 Summer Olympics')]
[('obj', 'http://www.wikidata.org/entity/Q8577'), ('name', '2012 Summer Olympics')]
5


In [8]:
# Lebron James part of (wdt:P361). Predicate found in a previous query. But is used for basketball player and not Lebron James -> empty set
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   # bind something
   wd:Q36159 wdt:P361 ?obj .
   # get the label
   ?obj sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
Empty


In [9]:
# Lebron James league (wdt:P118). Predicate found in the first query of the notebook.
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   # bind something
   wd:Q36159 wdt:P118 ?obj .
   # get the label
   ?obj sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q155223'), ('name', 'National Basketball Association')]
1


In [10]:
# properties of summer olympics 2012 (wd:Q8577). IRI found in a previous query.
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q8577 ?p ?obj .
   # get the label
   ?p sc:name ?name.
   FILTER (!isLiteral(?obj))
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P155'), ('name', 'follows')]
[('p', 'http://www.wikidata.org/prop/direct/P156'), ('name', 'followed by')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('name', 'country')]
[('p', 'http://www.wikidata.org/prop/direct/P18'), ('name', 'image')]
[('p', 'http://www.wikidata.org/prop/direct/P2670'), ('name', 'has parts of the class')]
[('p', 'http://www.wikidata.org/prop/direct/P276'), ('name', 'location')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of')]
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('name', 'has part')]
[('p', 'http://www.wikidata.org/prop/direct/P542'), ('name', 'officially opened by')]
[('p', 'http://www.wikidata.org/prop/direct/P543'), ('name', 'oath made by')]
[('p', 'http://www.wikidata.org/prop/direct/P545'), ('name', 'torch lit by')]
[('p', 'http://www.wikidata.org/prop/direct/P664'), ('name', 'organizer')]
[('p', 'http://www.wikidata.org/prop/direct/P710'), ('name', 'particip

In [11]:
# significant event (wdt:P793) of summer olympics 2012 (wd:Q8577)
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   # bind something
   wd:Q8577 wdt:P793 ?obj .
   # get the label
   ?obj sc:name ?name.

}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q1146933'), ('name', '2012 Summer Olympics closing ceremony')]
[('obj', 'http://www.wikidata.org/entity/Q168545'), ('name', '2012 Summer Olympics opening ceremony')]
[('obj', 'http://www.wikidata.org/entity/Q6671078'), ('name', 'London bid for the 2012 Summer Olympics')]
3


In [12]:
# not working... increase limit of Lebron James properties
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q36159 ?p ?obj .
   # get the label
   ?p sc:name ?name.
}
LIMIT 200
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('name', 'occupation')]
[('p', 'http://www.wikidata.org/prop/direct/P118'), ('name', 'league')]
[('p', 'http://www.wikidata.org/prop/direct/P1207'), ('name', 'NUKAT ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1263'), ('name', 'NNDB people ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1280'), ('name', 'CONOR.SI ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1285'), ('name', 'Munzinger Sport number')]
[('p', 'http://www.wikidata.org/prop/direct/P1344'), ('name', 'participant in')]
[('p', 'http://www.wikidata.org/prop/direct/P1411'), ('name', 'nominated for')]
[('p', 'http://www.wikidata.org/prop/direct/P1412'), ('name', 'languages spoken, written or signed')]
[('p', 'http://www.wikidata.org/prop/direct/P1417'), ('name', 'Encyclopædia Britannica Online ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1447'), ('name', 'Sports-Reference.com Olympic athlete ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1532'),

In [13]:
# Lebron James "member of the sports team" (wdt:P54)
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   # bind something
   wd:Q36159 wdt:P54 ?obj .
   # get the label
   ?obj sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q169138'), ('name', 'Miami Heat')]
[('obj', 'http://www.wikidata.org/entity/Q121783'), ('name', 'Los Angeles Lakers')]
[('obj', 'http://www.wikidata.org/entity/Q162990'), ('name', 'Cleveland Cavaliers')]
3


In [14]:
# FINAL QUERY
# Cleveland Cavaliers instance of
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   # bind something
   wd:Q162990 wdt:P31 ?obj .
   # get the label
   ?obj sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q13393265'), ('name', 'basketball team')]
1


>Cleveland Cavaliers is an instance of "wd:Q13393265" with label "basketball team"

## Task 4
Return all italian basketball teams born before 1945 (the result set must be a list of triples basketball team IRI, label and year of inception).

In [15]:
# baketball team nationality. Not helpful..
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   ?team wdt:P31 wd:Q13393265;
         wdt:P27 ?obj .
   # get the label
   ?obj sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q148'), ('name', "People's Republic of China")]
1


In [16]:
# basketball team properties. Not really what I should be looking for..
queryString = """
SELECT DISTINCT ?p  ?name
WHERE {
   wd:Q13393265 ?p ?obj.
   # get the label
   ?p sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1424'), ('name', "topic's main template")]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('name', 'subclass of')]
[('p', 'http://www.wikidata.org/prop/direct/P3417'), ('name', 'Quora topic ID')]
[('p', 'http://www.wikidata.org/prop/direct/P373'), ('name', 'Commons category')]
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('name', 'has part')]
[('p', 'http://www.wikidata.org/prop/direct/P641'), ('name', 'sport')]
[('p', 'http://www.wikidata.org/prop/direct/P8408'), ('name', 'KBpedia ID')]
[('p', 'http://www.wikidata.org/prop/direct/P910'), ('name', "topic's main category")]
8


In [17]:
# basketball team properties with some objects. Not really what I should be looking for..
queryString = """
SELECT DISTINCT ?p ?name ?obj ?objName
WHERE {
   wd:Q13393265 ?p ?obj.
   # get the label
   ?p sc:name ?name.
   ?obj sc:name ?objName.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('name', 'has part'), ('obj', 'http://www.wikidata.org/entity/Q51747567'), ('objName', 'squad')]
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('name', 'has part'), ('obj', 'http://www.wikidata.org/entity/Q3665646'), ('objName', 'basketball player')]
[('p', 'http://www.wikidata.org/prop/direct/P641'), ('name', 'sport'), ('obj', 'http://www.wikidata.org/entity/Q5372'), ('objName', 'basketball')]
[('p', 'http://www.wikidata.org/prop/direct/P527'), ('name', 'has part'), ('obj', 'http://www.wikidata.org/entity/Q5137571'), ('objName', 'basketball coach')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('name', 'subclass of'), ('obj', 'http://www.wikidata.org/entity/Q847017'), ('objName', 'sports club')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('name', 'subclass of'), ('obj', 'http://www.wikidata.org/entity/Q12973014'), ('objName', 'sports team')]
[('p', 'http://www.wikidata.org/prop/direct/P1424'), ('name', "

In [18]:
# properties of Cleveland Cavaliers
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q162990 ?p ?obj .
   # get the label
   ?p sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P115'), ('name', 'home venue')]
[('p', 'http://www.wikidata.org/prop/direct/P118'), ('name', 'league')]
[('p', 'http://www.wikidata.org/prop/direct/P127'), ('name', 'owned by')]
[('p', 'http://www.wikidata.org/prop/direct/P1417'), ('name', 'Encyclopædia Britannica Online ID')]
[('p', 'http://www.wikidata.org/prop/direct/P154'), ('name', 'logo image')]
[('p', 'http://www.wikidata.org/prop/direct/P159'), ('name', 'headquarters location')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('name', 'country')]
[('p', 'http://www.wikidata.org/prop/direct/P18'), ('name', 'image')]
[('p', 'http://www.wikidata.org/prop/direct/P2002'), ('name', 'Twitter username')]
[('p', 'http://www.wikidata.org/prop/direct/P214'), ('name', 'VIAF ID')]
[('p', 'http://www.wikidata.org/prop/direct/P244'), ('name', 'Library of Congress authority ID')]
[('p', 'http://www.wikidata.org/prop/direct/P286'), ('name', 'head coach')]
[('p', 'http://www.wikidata.org/pro

In [19]:
# all possible countries of basketball team, filtering Italy.
# Using predicate "country" (wdt:P17)
queryString = """
SELECT DISTINCT ?obj ?objName
WHERE {
    ?team wdt:P31 wd:Q13393265;
          wdt:P17 ?obj.
    # get the label
   ?obj sc:name ?objName.
    
    FILTER (str(?objName) = "Italy")

}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q38'), ('objName', 'Italy')]
1


In [20]:
# all possible properties of Italian basketball teams
queryString = """
SELECT DISTINCT ?p ?pName
WHERE {
    ?team wdt:P31 wd:Q13393265;
        wdt:P17 wd:Q38;
        ?p ?obj.
   # get the label
   ?p sc:name ?pName.
}
LIMIT 200
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P4003'), ('pName', 'Facebook page ID')]
[('p', 'http://www.wikidata.org/prop/direct/P115'), ('pName', 'home venue')]
[('p', 'http://www.wikidata.org/prop/direct/P118'), ('pName', 'league')]
[('p', 'http://www.wikidata.org/prop/direct/P127'), ('pName', 'owned by')]
[('p', 'http://www.wikidata.org/prop/direct/P131'), ('pName', 'located in the administrative territorial entity')]
[('p', 'http://www.wikidata.org/prop/direct/P1454'), ('pName', 'legal form')]
[('p', 'http://www.wikidata.org/prop/direct/P154'), ('pName', 'logo image')]
[('p', 'http://www.wikidata.org/prop/direct/P159'), ('pName', 'headquarters location')]
[('p', 'http://www.wikidata.org/prop/direct/P166'), ('pName', 'award received')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('pName', 'country')]
[('p', 'http://www.wikidata.org/prop/direct/P18'), ('pName', 'image')]
[('p', 'http://www.wikidata.org/prop/direct/P2002'), ('pName', 'Twitter username')]
[('p', 'http://w

In [21]:
# inception (wdt:P571) of some random Italian teams to see the object formatting (the date)
queryString = """
SELECT DISTINCT ?obj
WHERE {
    ?team wdt:P31 wd:Q13393265;
        wdt:P17 wd:Q38;
        wdt:P571 ?obj. 
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('obj', '1945-01-01T00:00:00Z')]
[('obj', '1970-01-01T00:00:00Z')]
[('obj', '1961-01-01T00:00:00Z')]
[('obj', '1948-01-01T00:00:00Z')]
[('obj', '1994-01-01T00:00:00Z')]
[('obj', '1974-01-01T00:00:00Z')]
[('obj', '1949-01-01T00:00:00Z')]
[('obj', '1951-01-01T00:00:00Z')]
[('obj', '1967-01-01T00:00:00Z')]
[('obj', '1975-01-01T00:00:00Z')]
[('obj', '1946-01-01T00:00:00Z')]
[('obj', '1966-01-01T00:00:00Z')]
[('obj', '1947-01-01T00:00:00Z')]
[('obj', '1943-01-01T00:00:00Z')]
[('obj', '1977-01-01T00:00:00Z')]
[('obj', '1960-01-01T00:00:00Z')]
[('obj', '1932-01-01T00:00:00Z')]
[('obj', '2011-01-01T00:00:00Z')]
[('obj', '1976-01-01T00:00:00Z')]
[('obj', '1978-01-01T00:00:00Z')]
20


In [22]:
# inception year of italian basketball teams before 1945
# Not really sure why it's including also teams born in 1945. The solution would be 
# FILTER (xsd:gYear(?date)<xsd:gYear("1944")) 
queryString = """
SELECT DISTINCT ?team ?teamLabel (year(?date) as ?inceptionYear)
WHERE {
    ?team wdt:P31 wd:Q13393265;
        wdt:P17 wd:Q38;
        wdt:P571 ?date.
   # get the label
   ?team sc:name ?teamLabel
   FILTER (xsd:gYear(?date)<xsd:gYear("1945"))
   

}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('team', 'http://www.wikidata.org/entity/Q16586535'), ('teamLabel', 'Pallacanestro Brindisi'), ('inceptionYear', '1945')]
[('team', 'http://www.wikidata.org/entity/Q3891980'), ('teamLabel', 'Nuova Pallacanestro Pavia'), ('inceptionYear', '1943')]
[('team', 'http://www.wikidata.org/entity/Q36551'), ('teamLabel', 'Pallacanestro Varese'), ('inceptionYear', '1945')]
[('team', 'http://www.wikidata.org/entity/Q1139737'), ('teamLabel', 'Fortitudo Pallacanestro Bologna'), ('inceptionYear', '1932')]
[('team', 'http://www.wikidata.org/entity/Q36362'), ('teamLabel', 'Olimpia Milano'), ('inceptionYear', '1936')]
[('team', 'http://www.wikidata.org/entity/Q36500'), ('teamLabel', 'Mens Sana Basket'), ('inceptionYear', '1861')]
[('team', 'http://www.wikidata.org/entity/Q36264'), ('teamLabel', 'Virtus Pallacanestro Bologna'), ('inceptionYear', '1927')]
[('team', 'http://www.wikidata.org/entity/Q36311'), ('teamLabel', 'Pallacanestro Cantù'), ('inceptionYear', '1936')]
[('team', 'http://www.wiki

In [23]:
# FINAL QUERY
# inception year of italian basketball teams before 1945
# Look at the comments of the previous query
queryString = """
SELECT DISTINCT ?team ?teamLabel (year(?date) as ?inceptionYear)
WHERE {
    ?team wdt:P31 wd:Q13393265;
        wdt:P17 wd:Q38;
        wdt:P571 ?date.
   # get the label
   ?team sc:name ?teamLabel
   FILTER (xsd:gYear(?date)<xsd:gYear("1944"))
   

}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('team', 'http://www.wikidata.org/entity/Q3891980'), ('teamLabel', 'Nuova Pallacanestro Pavia'), ('inceptionYear', '1943')]
[('team', 'http://www.wikidata.org/entity/Q1139737'), ('teamLabel', 'Fortitudo Pallacanestro Bologna'), ('inceptionYear', '1932')]
[('team', 'http://www.wikidata.org/entity/Q36362'), ('teamLabel', 'Olimpia Milano'), ('inceptionYear', '1936')]
[('team', 'http://www.wikidata.org/entity/Q36500'), ('teamLabel', 'Mens Sana Basket'), ('inceptionYear', '1861')]
[('team', 'http://www.wikidata.org/entity/Q36264'), ('teamLabel', 'Virtus Pallacanestro Bologna'), ('inceptionYear', '1927')]
[('team', 'http://www.wikidata.org/entity/Q36311'), ('teamLabel', 'Pallacanestro Cantù'), ('inceptionYear', '1936')]
[('team', 'http://www.wikidata.org/entity/Q7127695'), ('teamLabel', 'Pallacanestro Pavia'), ('inceptionYear', '1943')]
[('team', 'http://www.wikidata.org/entity/Q3754192'), ('teamLabel', 'Fulgor Omegna'), ('inceptionYear', '1937')]
[('team', 'http://www.wikidata.org/

>Results in the format "Team IRI" - "Team Label" - "Inception year"

## Task 5
Identify the BGP for NBA finals

In [24]:
# Cleveland Cavaliers league
queryString = """
SELECT DISTINCT ?league ?name
WHERE {
   # bind something
   wd:Q162990 wdt:P118 ?league .
   # get the label
   ?league sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('league', 'http://www.wikidata.org/entity/Q155223'), ('name', 'National Basketball Association')]
1


In [25]:
# NBA properties
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q155223 ?p ?obj.
   # get the label
   ?p sc:name ?name.
}
LIMIT 200
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P6118'), ('name', 'season ends')]
[('p', 'http://www.wikidata.org/prop/direct/P1132'), ('name', 'number of participants')]
[('p', 'http://www.wikidata.org/prop/direct/P1273'), ('name', 'CANTIC ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1346'), ('name', 'winner')]
[('p', 'http://www.wikidata.org/prop/direct/P1365'), ('name', 'replaces')]
[('p', 'http://www.wikidata.org/prop/direct/P1417'), ('name', 'Encyclopædia Britannica Online ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1424'), ('name', "topic's main template")]
[('p', 'http://www.wikidata.org/prop/direct/P154'), ('name', 'logo image')]
[('p', 'http://www.wikidata.org/prop/direct/P1687'), ('name', 'Wikidata property')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('name', 'country')]
[('p', 'http://www.wikidata.org/prop/direct/P18'), ('name', 'image')]
[('p', 'http://www.wikidata.org/prop/direct/P1830'), ('name', 'owner of')]
[('p', 'http://www.wikidata.org/prop

In [26]:
# NBA "season ends" (wdt:P6118), "winner" (wdt:P1346), "competition class" (wdt:P2094), "event interval" (wdt:P2257),
# "has part" (wdt:P527), "topic's main category" (wdt:P910), "history of topic" (wdt:P2184)
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   # bind something
   wd:Q155223 ?p ?obj.
   # get the label
   ?obj sc:name ?name.
   VALUES ?p {wdt:P6118 wdt:P1346 wdt:P2094 wdt:P2257 wdt:P527 wdt:P910 wdt:P2184}
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q238240'), ('name', 'Eastern Conference')]
[('obj', 'http://www.wikidata.org/entity/Q118'), ('name', 'April')]
[('obj', 'http://www.wikidata.org/entity/Q46343203'), ('name', "men's basketball")]
[('obj', 'http://www.wikidata.org/entity/Q121783'), ('name', 'Los Angeles Lakers')]
[('obj', 'http://www.wikidata.org/entity/Q318389'), ('name', 'Western Conference')]
[('obj', 'http://www.wikidata.org/entity/Q8663448'), ('name', 'Category:National Basketball Association')]
6


In [27]:
# From Los Angeles Lakers (wd:Q121783) see if I can go back to NBA finals
queryString = """
SELECT DISTINCT ?p ?pname ?obj ?name
WHERE {
   # bind something
   wd:Q121783 ?p ?obj .
   # get the label
   ?p sc:name ?pname.
   ?obj sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('pname', 'country'), ('obj', 'http://www.wikidata.org/entity/Q30'), ('name', 'United States of America')]
[('p', 'http://www.wikidata.org/prop/direct/P641'), ('pname', 'sport'), ('obj', 'http://www.wikidata.org/entity/Q5372'), ('name', 'basketball')]
[('p', 'http://www.wikidata.org/prop/direct/P159'), ('pname', 'headquarters location'), ('obj', 'http://www.wikidata.org/entity/Q65'), ('name', 'Los Angeles')]
[('p', 'http://www.wikidata.org/prop/direct/P118'), ('pname', 'league'), ('obj', 'http://www.wikidata.org/entity/Q155223'), ('name', 'National Basketball Association')]
[('p', 'http://www.wikidata.org/prop/direct/P1830'), ('pname', 'owner of'), ('obj', 'http://www.wikidata.org/entity/Q1781055'), ('name', 'South Bay Lakers')]
[('p', 'http://www.wikidata.org/prop/direct/P115'), ('pname', 'home venue'), ('obj', 'http://www.wikidata.org/entity/Q186080'), ('name', 'Staples Center')]
[('p', 'http://www.wikidata.org/prop/direct/P3

In [28]:
#see if "topic's main category" (wd:Q4417414) of Los Angeles Lakers leads to something. but it's a loop to Los Angeles Lakers wd:Q121783
queryString = """
SELECT DISTINCT ?p ?pname ?obj ?name
WHERE {
   # bind something
   wd:Q4417414 ?p ?obj .
   # get the label
   ?p sc:name ?pname.
   ?obj sc:name ?name.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pname', 'instance of'), ('obj', 'http://www.wikidata.org/entity/Q4167836'), ('name', 'Wikimedia category')]
[('p', 'http://www.wikidata.org/prop/direct/P301'), ('pname', "category's main topic"), ('obj', 'http://www.wikidata.org/entity/Q121783'), ('name', 'Los Angeles Lakers')]
2


In [29]:
# incoming edges to NBA. I am looking only for IRI's that have a label (sc:name)
# because NBA Finals should have one and not be a literal.
queryString = """
SELECT DISTINCT ?p ?pname
WHERE {
   # bind something
   ?sub ?p wd:Q155223.
   # get the label
   ?sub sc:name ?name.
   ?p sc:name ?pname.
}
LIMIT 200
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1591'), ('pname', 'defendant')]
[('p', 'http://www.wikidata.org/prop/direct/P1423'), ('pname', 'template has topic')]
[('p', 'http://www.wikidata.org/prop/direct/P101'), ('pname', 'field of work')]
[('p', 'http://www.wikidata.org/prop/direct/P108'), ('pname', 'employer')]
[('p', 'http://www.wikidata.org/prop/direct/P118'), ('pname', 'league')]
[('p', 'http://www.wikidata.org/prop/direct/P127'), ('pname', 'owned by')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('pname', 'described by source')]
[('p', 'http://www.wikidata.org/prop/direct/P1366'), ('pname', 'replaced by')]
[('p', 'http://www.wikidata.org/prop/direct/P137'), ('pname', 'operator')]
[('p', 'http://www.wikidata.org/prop/direct/P264'), ('pname', 'record label')]
[('p', 'http://www.wikidata.org/prop/direct/P301'), ('pname', "category's main topic")]
[('p', 'http://www.wikidata.org/prop/direct/P3450'), ('pname', 'sports season of league or competition')]
[('p', 'http

In [30]:
# "sports season of league or competition" (wdt:P3450) of NBA
queryString = """
SELECT DISTINCT ?sub ?name
WHERE {
   # bind something
   ?sub wdt:P3450 wd:Q155223.
   # get the label
   ?sub sc:name ?name.
   #?p sc:name ?pname.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('sub', 'http://www.wikidata.org/entity/Q600389'), ('name', '1990–91 NBA season')]
[('sub', 'http://www.wikidata.org/entity/Q2264531'), ('name', '1967–68 NBA season')]
[('sub', 'http://www.wikidata.org/entity/Q241834'), ('name', '1989–90 NBA season')]
[('sub', 'http://www.wikidata.org/entity/Q602553'), ('name', '1969–70 NBA season')]
[('sub', 'http://www.wikidata.org/entity/Q2294127'), ('name', '1971–72 NBA season')]
[('sub', 'http://www.wikidata.org/entity/Q113619'), ('name', '2012–13 NBA season')]
[('sub', 'http://www.wikidata.org/entity/Q16191100'), ('name', '2014-15 NBA season')]
[('sub', 'http://www.wikidata.org/entity/Q19862716'), ('name', '2015–16 NBA season')]
[('sub', 'http://www.wikidata.org/entity/Q631240'), ('name', '1979–80 NBA season')]
[('sub', 'http://www.wikidata.org/entity/Q1142320'), ('name', '2005–06 NBA season')]
[('sub', 'http://www.wikidata.org/entity/Q281094'), ('name', '1949–50 NBA season')]
[('sub', 'http://www.wikidata.org/entity/Q52702755'), ('name'

In [31]:
# incoming edges to 2017-18 NBA season (wdt:Q29098111), nothing helpful
queryString = """
SELECT DISTINCT ?p ?pname 
WHERE {
   # bind something
   ?sub ?p wd:Q29098111.
   # get the label
   #?sub sc:name ?name.
   ?p sc:name ?pname.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1344'), ('pname', 'participant in')]
[('p', 'http://www.wikidata.org/prop/direct/P301'), ('pname', "category's main topic")]
2


In [32]:
# predicates of 2017-18 NBA season with objects, nothing helpful
queryString = """
SELECT DISTINCT ?p ?pname ?obj ?objname
WHERE {
   # bind something
   wd:Q29098111 ?p ?obj .
   # get the label
   ?obj sc:name ?objname.
   ?p sc:name ?pname.
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1346'), ('pname', 'winner'), ('obj', 'http://www.wikidata.org/entity/Q157376'), ('objname', 'Golden State Warriors')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('pname', 'country'), ('obj', 'http://www.wikidata.org/entity/Q30'), ('objname', 'United States of America')]
[('p', 'http://www.wikidata.org/prop/direct/P2094'), ('pname', 'competition class'), ('obj', 'http://www.wikidata.org/entity/Q46343203'), ('objname', "men's basketball")]
[('p', 'http://www.wikidata.org/prop/direct/P2348'), ('pname', 'time period'), ('obj', 'http://www.wikidata.org/entity/Q47015413'), ('objname', '2017-2018 one-year-period')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pname', 'instance of'), ('obj', 'http://www.wikidata.org/entity/Q27020041'), ('objname', 'sports season')]
[('p', 'http://www.wikidata.org/prop/direct/P3450'), ('pname', 'sports season of league or competition'), ('obj', 'http://www.wikidata.org/entity/Q155223'), ('objn

In [33]:
# significant event of summer olympics 2012 (wd:Q8577). Summer Olympics not related to NBA Finals..
queryString = """
SELECT DISTINCT ?obj ?objname
WHERE {
   # bind something
   wd:Q8577 wdt:P793 ?obj .
   # get the label
   ?obj sc:name ?objname.

}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q1146933'), ('objname', '2012 Summer Olympics closing ceremony')]
[('obj', 'http://www.wikidata.org/entity/Q168545'), ('objname', '2012 Summer Olympics opening ceremony')]
[('obj', 'http://www.wikidata.org/entity/Q6671078'), ('objname', 'London bid for the 2012 Summer Olympics')]
3


In [34]:
# simple regex for NBA finals, but too complex with higher limits..
# This is a very inefficient query
queryString = """
SELECT DISTINCT ?s ?sname
WHERE {
   # bind something
   ?s ?p ?o.
   # get the label
   ?s sc:name ?sname.
   FILTER(REGEX(str(?sname),"NBA"))

}
LIMIT 2
"""

print("Results")
x=run_query(queryString)

Results
[('s', 'http://www.wikidata.org/prop/direct/P4718'), ('sname', 'Basketball-Reference.com NBA coach ID')]
[('s', 'http://www.wikidata.org/prop/direct/P3588'), ('sname', 'WNBA player ID')]
2


In [35]:
# Restart to look for NBA Finals from basketball
# incoming and outgoing edges of basketball
# I should have looked for objects that contain "final"..
queryString = """
SELECT DISTINCT ?edge ?edgeName 
WHERE {
        {
           # bind something
           wd:Q5372 ?edge ?object.
           # get the label
           ?edge sc:name ?edgeName.
            FILTER (!isLiteral(?object))
           }
           UNION
        {
           ?subject ?edge wd:Q5372.
           ?edge sc:name ?edgeName
           }
}
LIMIT 200
"""

print("Results")
x=run_query(queryString)

Results
[('edge', 'http://www.wikidata.org/prop/direct/P1151'), ('edgeName', "topic's main Wikimedia portal")]
[('edge', 'http://www.wikidata.org/prop/direct/P18'), ('edgeName', 'image')]
[('edge', 'http://www.wikidata.org/prop/direct/P1889'), ('edgeName', 'different from')]
[('edge', 'http://www.wikidata.org/prop/direct/P2094'), ('edgeName', 'competition class')]
[('edge', 'http://www.wikidata.org/prop/direct/P2283'), ('edgeName', 'uses')]
[('edge', 'http://www.wikidata.org/prop/direct/P279'), ('edgeName', 'subclass of')]
[('edge', 'http://www.wikidata.org/prop/direct/P2888'), ('edgeName', 'exact match')]
[('edge', 'http://www.wikidata.org/prop/direct/P2910'), ('edgeName', 'icon')]
[('edge', 'http://www.wikidata.org/prop/direct/P3095'), ('edgeName', 'practiced by')]
[('edge', 'http://www.wikidata.org/prop/direct/P31'), ('edgeName', 'instance of')]
[('edge', 'http://www.wikidata.org/prop/direct/P443'), ('edgeName', 'pronunciation audio')]
[('edge', 'http://www.wikidata.org/prop/direct/

In [36]:
# incoming and outgoing edges of NBA with objects that contain "final". None..
queryString = """
SELECT DISTINCT ?edge ?edgeName 
WHERE {
        {
           # bind something
           wd:Q155223 ?edge ?object.
           # get the label
           ?edge sc:name ?edgeName.
           ?object sc:name ?objectName.
           FILTER(REGEX(str(?objectName),"(?i)final"))
           FILTER (!isLiteral(?object))
        }UNION
        {
           ?subject ?edge wd:Q155223.
           ?edge sc:name ?edgeName.
           ?subject sc:name ?subjectName.
           FILTER(REGEX(str(?subjectName),"(?i)final"))
       }
}
LIMIT 200
"""

print("Results")
x=run_query(queryString)

Results
Empty


In [37]:
# if NBA finals exists I expect it to be connected to NBA with a short path,i.e. at most 3 traversals
# But nothing helpful...I was using LIMIT 2.. maybe that's why
# there is also an error (?subject (<>|!<>){,3} wd:Q155223.)

# !!!!
# Since I am asking for any path, it can be inefficient and take long to compute. I am doing it because I have a hard time finding 
# the exact path
# !!!!

queryString = """
SELECT DISTINCT ?iri ?iriName 
WHERE {
        {
            # bind something
            wd:Q155223 (<>|!<>){,3} ?iri.
            # get the label
            ?iri sc:name ?iriName.
            FILTER(REGEX(str(?iriName),"(?i)finals"))
            FILTER (!isLiteral(?iri))
       }UNION
       {
           ?subject (<>|!<>){,3} wd:Q155223.
           #?edge sc:name ?iriName.
           ?iri sc:name ?iriName.
           FILTER(REGEX(str(?iriName),"(?i)finals"))
       }
}
LIMIT 2
"""

print("Results")
x=run_query(queryString)



Results
[('iri', 'http://www.wikidata.org/entity/Q739499'), ('iriName', 'Bill Russell NBA Finals Most Valuable Player Award')]
[('iri', 'http://www.wikidata.org/entity/Q4016858'), ('iriName', '1976 World Championship Tennis Finals')]
2


In [38]:
# if NBA finals exists I expect it to be connected to basketball with a short path,e.g. at most 3 traversals
# This query can take some seconds to run..
# iteratively increased LIMIT and decreased path length
# I afterward realized I did a mistake which made the query return 2017 NBA Finals -> ("?subject (<>|!<>){,2} wd:5372.") instead of  ("?iri (<>|!<>){,2} wd:5372.") 
# So I found 2016 and 2017 NBA Finals by chance

# !!!!
# Since I am asking for any path, it can be inefficient and take long to compute. I am doing it because I have a hard time finding 
# the exact path
# !!!!

queryString = """
SELECT DISTINCT ?iri ?iriName 
WHERE {{
   # bind something
   wd:5372 (<>|!<>){,2} ?iri.
   # get the label
   ?iri sc:name ?iriName.
   FILTER(REGEX(str(?iriName),"(?i)nba final"))
   
   FILTER (!isLiteral(?iri))
   }UNION{
   ?subject (<>|!<>){,2} wd:5372.
   #?edge sc:name ?iriName.
   ?iri sc:name ?iriName.
   FILTER(REGEX(str(?iriName),"(?i)nba final"))
   }
}
LIMIT 2
"""

print("Results")
x=run_query(queryString)

Results
[('iri', 'http://www.wikidata.org/entity/Q24264147'), ('iriName', '2016 NBA Finals')]
[('iri', 'http://www.wikidata.org/entity/Q29096468'), ('iriName', '2017 NBA Finals')]
2


In [39]:
# FINAL QUERY
# Incoming and outgoing edges with subjects/objects "2017 NBA Finals" that contain "final".
queryString = """
SELECT DISTINCT ?edge ?edgeName ?iri ?iriName
WHERE {{
   # bind something
   wd:Q29096468 ?edge ?iri.
   # get the label
   ?edge sc:name ?edgeName.
   ?iri sc:name ?iriName.
   FILTER(REGEX(str(?iriName),"(?i)final"))
   
   FILTER (!isLiteral(?iri))
   }UNION{
   ?iri ?edge wd:Q29096468.
   ?edge sc:name ?edgeName.
   ?iri sc:name ?iriName.
   FILTER(REGEX(str(?iriName),"(?i)final"))
   }
}
LIMIT 200
"""

print("Results")
x=run_query(queryString)

Results
[('edge', 'http://www.wikidata.org/prop/direct/P155'), ('edgeName', 'follows'), ('iri', 'http://www.wikidata.org/entity/Q24264147'), ('iriName', '2016 NBA Finals')]
[('edge', 'http://www.wikidata.org/prop/direct/P31'), ('edgeName', 'instance of'), ('iri', 'http://www.wikidata.org/entity/Q842375'), ('iriName', 'NBA Finals')]
[('edge', 'http://www.wikidata.org/prop/direct/P156'), ('edgeName', 'followed by'), ('iri', 'http://www.wikidata.org/entity/Q24264147'), ('iriName', '2016 NBA Finals')]
3


>The node we are interesed in is "wd:Q842375" with label "NBA Finals"

In [40]:
# I want to better understand how basketball is connected with 2017 NBA Finals
# Reducing path length to at most 1. but exactly 1 does not work (<>|!<>){1} -> something is not really working as I thought
# trying also <>* as property, any path repeated zero or more times? 
# Actually here I was using a wrong version, with (?subject (<>|!<>){,1}  wd:5372.) instead of (?iri (<>|!<>){,1}  wd:5372.)


queryString = """
SELECT DISTINCT ?iri ?iriName ?w
WHERE {{
   # bind something
   wd:5372 (<>|!<>){,1} ?iri.
   # get the label
   ?iri sc:name ?iriName.
   FILTER(REGEX(str(?iriName),"(?i)nba final"))
   
   FILTER (!isLiteral(?iri))
   }UNION{
   ?subject (<>|!<>){,1}  wd:5372.
   #?edge sc:name ?iriName.
   ?iri sc:name ?iriName.
   FILTER(REGEX(str(?iriName),"(?i)nba final"))
   }
}
LIMIT 2
"""

print("Results")
x=run_query(queryString)

Results
[('iri', 'http://www.wikidata.org/entity/Q24264147'), ('iriName', '2016 NBA Finals')]
[('iri', 'http://www.wikidata.org/entity/Q29096468'), ('iriName', '2017 NBA Finals')]
2


In [41]:
# if NBA finals exists I expect it to be connected to basketball with a short path,i.e. at most 3 traversals
# can take some seconds..
# iteratively increased LIMIT and decreased path length
# I afterward realized I did a mistake which made the query return 2017 NBA Finals -> "?subject (<>|!<>){,2} wd:5372." instead of  "?iri (<>|!<>){,2} wd:5372." 
# restarting with ?iri instead of ?subject
# After some trial and error I restarted from NBA (wd:Q155223) instead of basketball (wd:5372) and I find "2012 NBA Finals"

queryString = """
SELECT DISTINCT ?iri ?iriName 
WHERE {{
   # bind something
   wd:Q155223 (<>|!<>){,2} ?iri.
   # get the label
   ?iri sc:name ?iriName.
   FILTER(REGEX(str(?iriName),"(?i)nba final"))
   
   FILTER (!isLiteral(?iri))
   }UNION{
   ?iri (<>|!<>){,2} wd:Q155223.
   #?edge sc:name ?iriName.
   ?iri sc:name ?iriName.
   FILTER(REGEX(str(?iriName),"(?i)nba final"))
   }
}
LIMIT 2
"""

print("Results")
x=run_query(queryString)

Results
[('iri', 'http://www.wikidata.org/entity/Q739499'), ('iriName', 'Bill Russell NBA Finals Most Valuable Player Award')]
[('iri', 'http://www.wikidata.org/entity/Q2628013'), ('iriName', '2012 NBA Finals')]
2


In [42]:
# From the previous query I delete the first part and get the same results, I know source and target nodes.
# From these I get the path(s) connecting 2012 NBA Finals and NBA.
# What I now know is that the following path exists: "2012 NBA Finals" - ("predicate I don't know") - x (variable) - predicate "winner" or "home venue" - y(variable) - "NBA Finals"

queryString = """
SELECT DISTINCT ?p ?pname
WHERE {

    wd:Q2628013 (<>|!<>){,2} wd:Q155223. # I don't really need this line
   #?edge sc:name ?iriName.
   #?iri sc:name ?iriName.
   #FILTER(REGEX(str(?iriName),"(?i)nba final"))


   wd:Q2628013 (<>|!<>) ?x .
    ?x ?p ?y .
    ?y (<>|!<>) wd:Q155223 .
    
    ?p sc:name ?pname .
   }

LIMIT 2
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1346'), ('pname', 'winner')]
[('p', 'http://www.wikidata.org/prop/direct/P115'), ('pname', 'home venue')]
2


In [43]:
# I want to to the same but for 2017 NBA Finals, instead of 2012 NBA Finals. 
# Increasing limit, but I don't find 2017 NBA Finals in the result set..
# 2017 NBA Finals must have a different path, so I am just going to find the path between "2012 NBA Finals" and "NBA Finals"
queryString = """
SELECT DISTINCT ?iri ?iriName 
WHERE {{
   # bind something
   wd:Q155223 (<>|!<>){,2} ?iri.
   # get the label
   ?iri sc:name ?iriName.
   FILTER(REGEX(str(?iriName),"(?i)nba final"))
   
   FILTER (!isLiteral(?iri))
   }UNION{
   ?iri (<>|!<>){,2} wd:Q155223.
   #?edge sc:name ?iriName.
   ?iri sc:name ?iriName.
   FILTER(REGEX(str(?iriName),"(?i)nba final"))
   }
}
LIMIT 35
"""

print("Results")
x=run_query(queryString)

Results
[('iri', 'http://www.wikidata.org/entity/Q739499'), ('iriName', 'Bill Russell NBA Finals Most Valuable Player Award')]
[('iri', 'http://www.wikidata.org/entity/Q2628013'), ('iriName', '2012 NBA Finals')]
[('iri', 'http://www.wikidata.org/entity/Q4589592'), ('iriName', '1994 NBA Finals')]
[('iri', 'http://www.wikidata.org/entity/Q4590500'), ('iriName', '1995 NBA Finals')]
[('iri', 'http://www.wikidata.org/entity/Q2552745'), ('iriName', '2007 NBA Finals')]
[('iri', 'http://www.wikidata.org/entity/Q4604623'), ('iriName', '2005 NBA Finals')]
[('iri', 'http://www.wikidata.org/entity/Q1006129'), ('iriName', '2003 NBA Finals')]
[('iri', 'http://www.wikidata.org/entity/Q16481622'), ('iriName', '2014 NBA Finals')]
[('iri', 'http://www.wikidata.org/entity/Q4594932'), ('iriName', '1999 NBA Finals')]
[('iri', 'http://www.wikidata.org/entity/Q170590'), ('iriName', '2006 NBA Finals')]
[('iri', 'http://www.wikidata.org/entity/Q13405328'), ('iriName', '2013 NBA Finals')]
[('iri', 'http://www.w

In [44]:
# From the previous query I delete the first part and get the same results, I know source and target nodes.
# From these I get the path(s) connecting 2012 NBA Finals and NBA.
# What I now know is the following path exists: "2012 NBA Finals" - ("predicate I don't know") - x (variable) - predicate "winner" or "home venue" - y(variable) - "NBA Finals"
# returning the complete path between "2012 NBA Finals" and "NBA"



queryString = """
SELECT DISTINCT ?2012NBAFinals ?iname ?xname ?pname ?yname ?lname ?NBA
WHERE {

    wd:Q2628013 (<>|!<>){,2} wd:Q155223. # I don't really need this line
   #?edge sc:name ?iriName.
   #?iri sc:name ?iriName.
   #FILTER(REGEX(str(?iriName),"(?i)nba final"))


   wd:Q2628013 ?i ?x .
    ?x ?p ?y .
    ?y ?l wd:Q155223 .
    
    wd:Q2628013 sc:name ?2012NBAFinals.
    wd:Q155223 sc:name ?NBA.
    
    ?x sc:name ?xname .
    ?y sc:name ?yname .
    ?p sc:name ?pname .
    ?i sc:name ?iname .
    ?l sc:name ?lname .
   }

LIMIT 10
"""

print("Results")
x=run_query(queryString)

Results
[('2012NBAFinals', '2012 NBA Finals'), ('iname', 'followed by'), ('xname', '2013 NBA Finals'), ('pname', 'winner'), ('yname', 'Miami Heat'), ('lname', 'league'), ('NBA', 'National Basketball Association')]
[('2012NBAFinals', '2012 NBA Finals'), ('iname', 'participating team'), ('xname', 'Oklahoma City Thunder'), ('pname', 'home venue'), ('yname', 'Chesapeake Energy Arena'), ('lname', 'occupant'), ('NBA', 'National Basketball Association')]
2


## Task 6
Consider only the basketball teams which Kevin Durant and Lebron James played. Return the number of NBA finals won by these teams (the result set must be a list of triples basketball team IRI, label and #victories. Return a number for each basketball team).

In [45]:
#Miami Heat properies. no winner predicates. so going from NBA Season to winner
queryString = """
SELECT DISTINCT ?p ?name 
WHERE {
   # bind something
   wd:Q169138 ?p ?team .
   # get the label
   ?p sc:name ?name.
}
LIMIT 2000
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P115'), ('name', 'home venue')]
[('p', 'http://www.wikidata.org/prop/direct/P118'), ('name', 'league')]
[('p', 'http://www.wikidata.org/prop/direct/P127'), ('name', 'owned by')]
[('p', 'http://www.wikidata.org/prop/direct/P1417'), ('name', 'Encyclopædia Britannica Online ID')]
[('p', 'http://www.wikidata.org/prop/direct/P159'), ('name', 'headquarters location')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('name', 'country')]
[('p', 'http://www.wikidata.org/prop/direct/P18'), ('name', 'image')]
[('p', 'http://www.wikidata.org/prop/direct/P2002'), ('name', 'Twitter username')]
[('p', 'http://www.wikidata.org/prop/direct/P214'), ('name', 'VIAF ID')]
[('p', 'http://www.wikidata.org/prop/direct/P244'), ('name', 'Library of Congress authority ID')]
[('p', 'http://www.wikidata.org/prop/direct/P286'), ('name', 'head coach')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('name', 'instance of')]
[('p', 'http://www.wikidata.org/pro

In [46]:
# FINAL QUERY
# returning winners of nba seasons, where Lebron James and Keving Durant played

# Since NBA Finals is the championship to determine the league champion, I directly consider the winners of the leagues
# Also, from the previous task I didn't find a way to go back to each NBA Finals edition by using the same path. 

queryString = """
SELECT DISTINCT ?winner ?winnerName (COUNT(DISTINCT ?sub) as ?victories) 
WHERE {
   # bind something
   ?sub wdt:P3450 wd:Q155223.
   ?sub wdt:P1346 ?winner.
   ?player wdt:P54 ?winner.
     
  VALUES ?player {wd:Q36159 wd:Q29545}
  


   ?sub sc:name ?name.
   ?winner sc:name ?winnerName
}
LIMIT 200
"""

print("Results")
x=run_query(queryString)

Results
[('winner', 'http://www.wikidata.org/entity/Q157376'), ('winnerName', 'Golden State Warriors'), ('victories', '6')]
[('winner', 'http://www.wikidata.org/entity/Q121783'), ('winnerName', 'Los Angeles Lakers'), ('victories', '17')]
[('winner', 'http://www.wikidata.org/entity/Q235326'), ('winnerName', 'Seattle SuperSonics'), ('victories', '1')]
[('winner', 'http://www.wikidata.org/entity/Q169138'), ('winnerName', 'Miami Heat'), ('victories', '3')]
[('winner', 'http://www.wikidata.org/entity/Q162990'), ('winnerName', 'Cleveland Cavaliers'), ('victories', '1')]
5


>Results in the format "Team IRI" - "Team Label" - "number of victories"

In [47]:
# For some editions of NBA Finals I look if they have a winning predicate
queryString = """
SELECT DISTINCT ?NBAFinalsEdition ?predicate ?predicateName ?objectName
WHERE {
   # bind something
   ?NBAFinalsEdition ?predicate ?object.
     
  VALUES ?NBAFinalsEdition {wd:Q644353 wd:Q1320332 wd:Q1959941}
  


   ?predicate sc:name ?predicateName.
   ?object sc:name ?objectName
}
LIMIT 200
"""

print("Results")
x=run_query(queryString)

Results
[('NBAFinalsEdition', 'http://www.wikidata.org/entity/Q644353'), ('predicate', 'http://www.wikidata.org/prop/direct/P17'), ('predicateName', 'country'), ('objectName', 'United States of America')]
[('NBAFinalsEdition', 'http://www.wikidata.org/entity/Q1320332'), ('predicate', 'http://www.wikidata.org/prop/direct/P17'), ('predicateName', 'country'), ('objectName', 'United States of America')]
[('NBAFinalsEdition', 'http://www.wikidata.org/entity/Q1959941'), ('predicate', 'http://www.wikidata.org/prop/direct/P641'), ('predicateName', 'sport'), ('objectName', 'basketball')]
[('NBAFinalsEdition', 'http://www.wikidata.org/entity/Q644353'), ('predicate', 'http://www.wikidata.org/prop/direct/P641'), ('predicateName', 'sport'), ('objectName', 'basketball')]
[('NBAFinalsEdition', 'http://www.wikidata.org/entity/Q1320332'), ('predicate', 'http://www.wikidata.org/prop/direct/P641'), ('predicateName', 'sport'), ('objectName', 'basketball')]
[('NBAFinalsEdition', 'http://www.wikidata.org/en

In [48]:
# another way to do the final query.
# Still I trust more the previous approach (i.e. from NBA Season to winner) than this onw (i.e. from NBA Finals to winner),
# because from the previous task it seems to me that instances of NBA Finals are not populated enough. 
queryString = """
SELECT DISTINCT ?winner ?winnerName (COUNT(DISTINCT ?NBAFinals) as ?victories) 
WHERE {
   # bind something
   ?winner wdt:P118 wd:Q155223.
   ?NBAFinals wdt:P1346 ?winner.
   ?NBAFinals wdt:P31 wd:Q842375.
   ?player wdt:P54 ?winner.
     
  VALUES ?player {wd:Q36159 wd:Q29545}
  

   ?winner sc:name ?winnerName
}

"""

print("Results")
x=run_query(queryString)

Results
[('winner', 'http://www.wikidata.org/entity/Q121783'), ('winnerName', 'Los Angeles Lakers'), ('victories', '16')]
[('winner', 'http://www.wikidata.org/entity/Q157376'), ('winnerName', 'Golden State Warriors'), ('victories', '4')]
[('winner', 'http://www.wikidata.org/entity/Q169138'), ('winnerName', 'Miami Heat'), ('victories', '2')]
3


## Task 7
Consider only NBA (National Basketball Association) basketball teams and all basketball players who played in one of these teams at least once. Also consider the country of citizenship of the players with the related continent. Return for each continent the number of players who played in an NBA basketball teams. (the result set must be triples of continent IRI, labels and #players)

In [49]:
# examples of NBA player nationality
queryString = """
SELECT DISTINCT ?nationality ?nationalityName
WHERE {
   # bind something
   ?team wdt:P118 wd:Q155223.
   ?player wdt:P54 ?team;
           wdt:P27 ?nationality.
    ?nationality sc:name ?nationalityName.
    
}
LIMIT 5
"""

print("Results")
x=run_query(queryString)

Results
[('nationality', 'http://www.wikidata.org/entity/Q1033'), ('nationalityName', 'Nigeria')]
[('nationality', 'http://www.wikidata.org/entity/Q1041'), ('nationalityName', 'Senegal')]
[('nationality', 'http://www.wikidata.org/entity/Q142'), ('nationalityName', 'France')]
[('nationality', 'http://www.wikidata.org/entity/Q159'), ('nationalityName', 'Russia')]
[('nationality', 'http://www.wikidata.org/entity/Q16'), ('nationalityName', 'Canada')]
5


In [50]:
# outgoing edges from Nigeria, to check if belongs to a continent. continent predicate found (wdt:P30)
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
       wd:Q1033 ?p ?obj.
       ?p sc:name ?name.
       FILTER(!isLiteral(?obj))
    
}
LIMIT 100
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1151'), ('name', "topic's main Wikimedia portal")]
[('p', 'http://www.wikidata.org/prop/direct/P122'), ('name', 'basic form of government')]
[('p', 'http://www.wikidata.org/prop/direct/P1304'), ('name', 'central bank')]
[('p', 'http://www.wikidata.org/prop/direct/P1313'), ('name', 'office held by head of government')]
[('p', 'http://www.wikidata.org/prop/direct/P1343'), ('name', 'described by source')]
[('p', 'http://www.wikidata.org/prop/direct/P1365'), ('name', 'replaces')]
[('p', 'http://www.wikidata.org/prop/direct/P138'), ('name', 'named after')]
[('p', 'http://www.wikidata.org/prop/direct/P1464'), ('name', 'category for people born here')]
[('p', 'http://www.wikidata.org/prop/direct/P1465'), ('name', 'category for people who died here')]
[('p', 'http://www.wikidata.org/prop/direct/P150'), ('name', 'contains administrative territorial entity')]
[('p', 'http://www.wikidata.org/prop/direct/P155'), ('name', 'follows')]
[('p', 'http

In [51]:
# FINAL QUERY
# number of players per continent. 
# Strange continent names, next I'll look for continent IRI instead of predicate. 
queryString = """
SELECT DISTINCT ?continent ?continentName (COUNT(DISTINCT ?player) as ?players)
WHERE {
   # bind something
   ?team wdt:P118 wd:Q155223.
   ?player wdt:P54 ?team;
           wdt:P27 ?nationality.
    ?nationality wdt:P30 ?continent.
    
    
    ?continent sc:name ?continentName.
    ?nationality sc:name ?nationalityName.
    
}
GROUP BY ?continent ?continentName
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('continent', 'http://www.wikidata.org/entity/Q46'), ('continentName', 'Europe'), ('players', '304')]
[('continent', 'http://www.wikidata.org/entity/Q538'), ('continentName', 'Insular Oceania'), ('players', '3799')]
[('continent', 'http://www.wikidata.org/entity/Q15'), ('continentName', 'Africa'), ('players', '61')]
[('continent', 'http://www.wikidata.org/entity/Q48'), ('continentName', 'Asia'), ('players', '73')]
[('continent', 'http://www.wikidata.org/entity/Q5401'), ('continentName', 'Eurasia'), ('players', '17')]
[('continent', 'http://www.wikidata.org/entity/Q18'), ('continentName', 'South America'), ('players', '37')]
[('continent', 'http://www.wikidata.org/entity/Q49'), ('continentName', 'North America'), ('players', '3849')]
7


>Results in the format "Continent IRI" - "Continent Label" - "Number of players"

In [52]:
#  Asia (wd:Q48) instance of continent (wd:Q5107).
queryString = """
SELECT DISTINCT ?continent ?continentName
WHERE {
   # bind something
   wd:Q48 wdt:P31 ?continent.
   ?continent sc:name ?continentName.
    
}
LIMIT 5
"""

print("Results")
x=run_query(queryString)

Results
[('continent', 'http://www.wikidata.org/entity/Q5107'), ('continentName', 'continent')]
[('continent', 'http://www.wikidata.org/entity/Q2418896'), ('continentName', 'part of the world')]
2


In [53]:
# number of players per continent. 
# Less continents than before. Perhaps wdt:P30 is the standard IRI to indicate continents.
# It's wrong because using wdt:P361 for no apparent reason

queryString = """
SELECT DISTINCT (GROUP_CONCAT(DISTINCT ?countryName; separator=" ") AS ?countries) ?continent ?continentName (COUNT(?player) as ?players)
WHERE {
   # bind something
   ?team wdt:P118 wd:Q155223.
   ?player wdt:P54 ?team;
           wdt:P27 ?country.
    ?country wdt:P361 ?continent.
    ?continent wdt:P31 wd:Q5107.
    
    
    ?continent sc:name ?continentName.
    ?country sc:name ?countryName.
    
}
GROUP BY ?continent ?continentName
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('countries', 'Belgium Italy Norway Spain Switzerland'), ('continent', 'http://www.wikidata.org/entity/Q46'), ('continentName', 'Europe'), ('players', '108')]
[('countries', 'Argentina Brazil Colombia Guyana Venezuela'), ('continent', 'http://www.wikidata.org/entity/Q18'), ('continentName', 'South America'), ('players', '71')]
[('countries', 'Canada Mexico United States of America'), ('continent', 'http://www.wikidata.org/entity/Q49'), ('continentName', 'North America'), ('players', '9430')]
3


In [54]:
# number of players per continent. 

queryString = """
SELECT DISTINCT ?continent ?continentName (COUNT(DISTINCT ?player) as ?players) #(GROUP_CONCAT(DISTINCT ?countryName; separator=" ") AS ?countries) 
WHERE {
   # bind something
   ?team wdt:P118 wd:Q155223.
   ?player wdt:P54 ?team;
           wdt:P27 ?country.
    ?country ?referredInAnyWayTo ?continent.
    ?continent wdt:P31 wd:Q5107.
    
    
    ?continent sc:name ?continentName.
    ?country sc:name ?countryName.
    
}
GROUP BY ?continent ?continentName
LIMIT 50
"""

print("Results")
x=run_query(queryString)

Results
[('continent', 'http://www.wikidata.org/entity/Q46'), ('continentName', 'Europe'), ('players', '304')]
[('continent', 'http://www.wikidata.org/entity/Q538'), ('continentName', 'Insular Oceania'), ('players', '3799')]
[('continent', 'http://www.wikidata.org/entity/Q3960'), ('continentName', 'Australia'), ('players', '25')]
[('continent', 'http://www.wikidata.org/entity/Q15'), ('continentName', 'Africa'), ('players', '66')]
[('continent', 'http://www.wikidata.org/entity/Q48'), ('continentName', 'Asia'), ('players', '73')]
[('continent', 'http://www.wikidata.org/entity/Q18'), ('continentName', 'South America'), ('players', '37')]
[('continent', 'http://www.wikidata.org/entity/Q49'), ('continentName', 'North America'), ('players', '3849')]
[('continent', 'http://www.wikidata.org/entity/Q828'), ('continentName', 'Americas'), ('players', '3777')]
8


## Task 8
Identify the BGP for the NBA Most Valuable Player Award (MVP) (see https://en.wikipedia.org/wiki/NBA_Most_Valuable_Player_Award)

In [55]:
# NBA MVP is an annual award given for each season
# incoming/outgoing edges to/from 2017-18 NBA season, looking for mvp
# no award or winner predicate
queryString = """
SELECT DISTINCT ?p ?pname 
WHERE {{
   # bind something
   ?sub ?p wd:Q29098111.
   # get the label
   ?p sc:name ?pname.}
   UNION{
   wd:Q29098111 ?p ?obj.
   ?p sc:name ?pname.
   }
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1344'), ('pname', 'participant in')]
[('p', 'http://www.wikidata.org/prop/direct/P301'), ('pname', "category's main topic")]
[('p', 'http://www.wikidata.org/prop/direct/P1132'), ('pname', 'number of participants')]
[('p', 'http://www.wikidata.org/prop/direct/P1346'), ('pname', 'winner')]
[('p', 'http://www.wikidata.org/prop/direct/P1350'), ('pname', 'number of matches played/races/starts')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('pname', 'country')]
[('p', 'http://www.wikidata.org/prop/direct/P2094'), ('pname', 'competition class')]
[('p', 'http://www.wikidata.org/prop/direct/P2348'), ('pname', 'time period')]
[('p', 'http://www.wikidata.org/prop/direct/P2671'), ('pname', 'Google Knowledge Graph ID')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pname', 'instance of')]
[('p', 'http://www.wikidata.org/prop/direct/P3417'), ('pname', 'Quora topic ID')]
[('p', 'http://www.wikidata.org/prop/direct/P3450'), ('pname', '

In [56]:
# maybe mvb is connected with the player (Lebron James has for sure won an MVP)
queryString = """
SELECT DISTINCT ?p ?name
WHERE {
   # bind something
   wd:Q36159 ?p ?obj .
   # get the label
   ?p sc:name ?name.
}
LIMIT 200
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('name', 'occupation')]
[('p', 'http://www.wikidata.org/prop/direct/P118'), ('name', 'league')]
[('p', 'http://www.wikidata.org/prop/direct/P1207'), ('name', 'NUKAT ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1263'), ('name', 'NNDB people ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1280'), ('name', 'CONOR.SI ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1285'), ('name', 'Munzinger Sport number')]
[('p', 'http://www.wikidata.org/prop/direct/P1344'), ('name', 'participant in')]
[('p', 'http://www.wikidata.org/prop/direct/P1411'), ('name', 'nominated for')]
[('p', 'http://www.wikidata.org/prop/direct/P1412'), ('name', 'languages spoken, written or signed')]
[('p', 'http://www.wikidata.org/prop/direct/P1417'), ('name', 'Encyclopædia Britannica Online ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1447'), ('name', 'Sports-Reference.com Olympic athlete ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1532'),

In [57]:
# FINAL QUERY
# award received (wdt:P166) by Lebron James -> nba mvp wd:Q222047
queryString = """
SELECT DISTINCT ?obj ?name
WHERE {
   # bind something
   wd:Q36159 wdt:P166 ?obj .
   # get the label
   ?obj sc:name ?name.
}
LIMIT 200
"""

print("Results")
x=run_query(queryString)

Results
[('obj', 'http://www.wikidata.org/entity/Q1501719'), ('name', 'J. Walter Kennedy Citizenship Award')]
[('obj', 'http://www.wikidata.org/entity/Q4896605'), ('name', 'Best Championship Performance ESPY Award')]
[('obj', 'http://www.wikidata.org/entity/Q4896598'), ('name', 'Best Breakthrough Athlete ESPY Award')]
[('obj', 'http://www.wikidata.org/entity/Q743122'), ('name', 'Associated Press Athlete of the Year')]
[('obj', 'http://www.wikidata.org/entity/Q1418009'), ('name', 'NBA All-Rookie Team')]
[('obj', 'http://www.wikidata.org/entity/Q1465181'), ('name', 'NBA All-Defensive Team')]
[('obj', 'http://www.wikidata.org/entity/Q1653207'), ('name', 'Sports Illustrated Sportsperson of the Year')]
[('obj', 'http://www.wikidata.org/entity/Q222047'), ('name', 'NBA Most Valuable Player Award')]
[('obj', 'http://www.wikidata.org/entity/Q31391'), ('name', 'NBA All-Star Game Kobe Bryant Most Valuable Player Award')]
[('obj', 'http://www.wikidata.org/entity/Q4835465'), ('name', 'BET Award for

>The node we are interested in is "wd:Q222047" with label "NBA Most Valuable Player Award"

## Task 9
Consider the players who won the NBA Most Valuable Player Award. Return for these players the number of NBA Most Valuable Player Award won in their career (the result set must be triples of player IRI, label and #victories)

In [58]:
# incoming/outgoing edges to/from NBA mvp. No predicates refeering to date, with the exception with inception.
queryString = """
SELECT DISTINCT ?p ?pname 
WHERE {{
   # bind something
   ?sub ?p wd:Q222047.
   # get the label
   ?p sc:name ?pname.}
   UNION{
   wd:Q222047 ?p ?obj.
   OPTIONAL {?p sc:name ?pname.}
   }
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1411'), ('pname', 'nominated for')]
[('p', 'http://www.wikidata.org/prop/direct/P166'), ('pname', 'award received')]
[('p', 'http://www.wikidata.org/prop/direct/P301'), ('pname', "category's main topic")]
[('p', 'http://www.wikidata.org/prop/direct/P1346'), ('pname', 'winner')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('pname', 'country')]
[('p', 'http://www.wikidata.org/prop/direct/P1814'), ('pname', 'name in kana')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pname', 'instance of')]
[('p', 'http://www.wikidata.org/prop/direct/P571'), ('pname', 'inception')]
[('p', 'http://www.wikidata.org/prop/direct/P641'), ('pname', 'sport')]
[('p', 'http://www.wikidata.org/prop/direct/P646'), ('pname', 'Freebase ID')]
[('p', 'http://www.wikidata.org/prop/direct/P910'), ('pname', "topic's main category")]
[('p', 'http://www.w3.org/1999/02/22-rdf-syntax-ns#type')]
[('p', 'http://www.wikidata.org/prop/direct-normalized/P646')]
[(

In [59]:
# No specific instances of mvp award (e.g. no "2017 MVP Award")

queryString = """
SELECT DISTINCT ?p ?pname 
WHERE {
   # bind something
   ?sub wdt:P31 wd:Q222047.
   # get the label
   ?p sc:name ?pname.
  
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
Empty


In [60]:
# players that have as "award received" (wd:P166) an NBA MVP
queryString = """
SELECT DISTINCT ?sub ?name 
WHERE {
   # bind something
   ?sub wdt:P166 wd:Q222047.
   # get the label
   ?sub sc:name ?name.
   #?p sc:name ?pname.

}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('sub', 'http://www.wikidata.org/entity/Q179051'), ('name', 'Kareem Abdul-Jabbar')]
[('sub', 'http://www.wikidata.org/entity/Q134183'), ('name', 'Magic Johnson')]
[('sub', 'http://www.wikidata.org/entity/Q169452'), ('name', "Shaquille O'Neal")]
[('sub', 'http://www.wikidata.org/entity/Q17717'), ('name', 'Oscar Robertson')]
[('sub', 'http://www.wikidata.org/entity/Q182455'), ('name', 'Wilt Chamberlain')]
[('sub', 'http://www.wikidata.org/entity/Q189240'), ('name', 'Kevin Garnett')]
[('sub', 'http://www.wikidata.org/entity/Q190094'), ('name', 'Allen Iverson')]
[('sub', 'http://www.wikidata.org/entity/Q190152'), ('name', 'Larry Bird')]
[('sub', 'http://www.wikidata.org/entity/Q192707'), ('name', 'Charles Barkley')]
[('sub', 'http://www.wikidata.org/entity/Q193020'), ('name', 'Tim Duncan')]
[('sub', 'http://www.wikidata.org/entity/Q196185'), ('name', 'Russell Westbrook')]
[('sub', 'http://www.wikidata.org/entity/Q199896'), ('name', 'Karl Malone')]
[('sub', 'http://www.wikidata.org

In [61]:
# outgoing edges from NBA mvp with objects. Looking for year of the victory. 

queryString = """
SELECT DISTINCT ?p ?pname ?obj ?name
WHERE {
   wd:Q222047 ?p ?obj.
   ?obj sc:name ?name.
   ?p sc:name ?pname.
   
}
LIMIT 200
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1346'), ('pname', 'winner'), ('obj', 'http://www.wikidata.org/entity/Q29545'), ('name', 'Kevin Durant')]
[('p', 'http://www.wikidata.org/prop/direct/P1346'), ('pname', 'winner'), ('obj', 'http://www.wikidata.org/entity/Q205326'), ('name', 'Derrick Rose')]
[('p', 'http://www.wikidata.org/prop/direct/P1346'), ('pname', 'winner'), ('obj', 'http://www.wikidata.org/entity/Q192707'), ('name', 'Charles Barkley')]
[('p', 'http://www.wikidata.org/prop/direct/P1346'), ('pname', 'winner'), ('obj', 'http://www.wikidata.org/entity/Q192955'), ('name', 'Steve Nash')]
[('p', 'http://www.wikidata.org/prop/direct/P1346'), ('pname', 'winner'), ('obj', 'http://www.wikidata.org/entity/Q182455'), ('name', 'Wilt Chamberlain')]
[('p', 'http://www.wikidata.org/prop/direct/P1346'), ('pname', 'winner'), ('obj', 'http://www.wikidata.org/entity/Q380515'), ('name', 'Willis Reed')]
[('p', 'http://www.wikidata.org/prop/direct/P1346'), ('pname', 'winner'), ('obj', '

In [62]:
# incoming edges to NBA mvp
queryString = """
SELECT DISTINCT ?sub ?name ?p ?pname 
WHERE {
   ?sub ?p wd:Q222047.
   ?sub sc:name ?name.
   ?p sc:name ?pname.
   
}
LIMIT 200
"""

print("Results")
x=run_query(queryString)

Results
[('sub', 'http://www.wikidata.org/entity/Q352696'), ('name', 'Michael Beasley'), ('p', 'http://www.wikidata.org/prop/direct/P1411'), ('pname', 'nominated for')]
[('sub', 'http://www.wikidata.org/entity/Q264393'), ('name', 'Dave Cowens'), ('p', 'http://www.wikidata.org/prop/direct/P166'), ('pname', 'award received')]
[('sub', 'http://www.wikidata.org/entity/Q190152'), ('name', 'Larry Bird'), ('p', 'http://www.wikidata.org/prop/direct/P166'), ('pname', 'award received')]
[('sub', 'http://www.wikidata.org/entity/Q553232'), ('name', 'Bob McAdoo'), ('p', 'http://www.wikidata.org/prop/direct/P166'), ('pname', 'award received')]
[('sub', 'http://www.wikidata.org/entity/Q273256'), ('name', 'Hakeem Olajuwon'), ('p', 'http://www.wikidata.org/prop/direct/P166'), ('pname', 'award received')]
[('sub', 'http://www.wikidata.org/entity/Q17717'), ('name', 'Oscar Robertson'), ('p', 'http://www.wikidata.org/prop/direct/P166'), ('pname', 'award received')]
[('sub', 'http://www.wikidata.org/entity/

In [63]:
# outgoing edges from NBA mvp with objects. Looking for some year references..
# Only date present is the inception, likely of the award itself
queryString = """
SELECT DISTINCT ?p ?pname ?obj ?name
WHERE {
   wd:Q222047 ?p ?obj.
   ?p sc:name ?pname.
   OPTIONAL {?obj sc:name ?name.}
   
}
LIMIT 200
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1346'), ('pname', 'winner'), ('obj', 'http://www.wikidata.org/entity/Q179051'), ('name', 'Kareem Abdul-Jabbar')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('pname', 'country'), ('obj', 'http://www.wikidata.org/entity/Q30'), ('name', 'United States of America')]
[('p', 'http://www.wikidata.org/prop/direct/P641'), ('pname', 'sport'), ('obj', 'http://www.wikidata.org/entity/Q5372'), ('name', 'basketball')]
[('p', 'http://www.wikidata.org/prop/direct/P1346'), ('pname', 'winner'), ('obj', 'http://www.wikidata.org/entity/Q134183'), ('name', 'Magic Johnson')]
[('p', 'http://www.wikidata.org/prop/direct/P1346'), ('pname', 'winner'), ('obj', 'http://www.wikidata.org/entity/Q17717'), ('name', 'Oscar Robertson')]
[('p', 'http://www.wikidata.org/prop/direct/P1346'), ('pname', 'winner'), ('obj', 'http://www.wikidata.org/entity/Q182455'), ('name', 'Wilt Chamberlain')]
[('p', 'http://www.wikidata.org/prop/direct/P1346'), ('pname', 'winner'

In [64]:
# NBA MVP is an annual award given for each season
# incoming/outgoing edges to/from 2017-18 NBA season, looking if there is a direct path to MVP.
# Empty result set -> no way to reference and mvp to a given season 
# This query actually doesn't have much sense

queryString = """
SELECT DISTINCT ?p 
WHERE {{
   # bind something
   wd:Q222047 ?p wd:Q29098111.}
   UNION{
   wd:Q29098111 ?p wd:Q222047.
   }
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
Empty


In [65]:
# Looking to all objects of 2017-18 NBA Season, maybe there exists an mvp predicate

queryString = """
SELECT DISTINCT ?p ?pname ?o ?oname
WHERE {
   # bind something
   wd:Q29098111 ?p ?o .
   ?p sc:name ?pname .
   ?o sc:name ?oname .

   
}
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('pname', 'instance of'), ('o', 'http://www.wikidata.org/entity/Q27020041'), ('oname', 'sports season')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('pname', 'country'), ('o', 'http://www.wikidata.org/entity/Q30'), ('oname', 'United States of America')]
[('p', 'http://www.wikidata.org/prop/direct/P2094'), ('pname', 'competition class'), ('o', 'http://www.wikidata.org/entity/Q46343203'), ('oname', "men's basketball")]
[('p', 'http://www.wikidata.org/prop/direct/P641'), ('pname', 'sport'), ('o', 'http://www.wikidata.org/entity/Q5372'), ('oname', 'basketball')]
[('p', 'http://www.wikidata.org/prop/direct/P664'), ('pname', 'organizer'), ('o', 'http://www.wikidata.org/entity/Q155223'), ('oname', 'National Basketball Association')]
[('p', 'http://www.wikidata.org/prop/direct/P3450'), ('pname', 'sports season of league or competition'), ('o', 'http://www.wikidata.org/entity/Q155223'), ('oname', 'National Basketball Association

> MVP Award is directly connected with the winners. There are not different instances for each year/edition of the award.
> 2017-18 NBA Season does not have an outgoing predicate such as "MVP award won by" and the player name. 

### Since I have no direct NBA MVP information from the NBA Season, and no direct time information from the NBA MVP award, I conclude it's not possible to find the number of MVP awards for each player. I think it's a reification in Wikidata, because I need a statement (year) about the MVP award winner. 
